## Setting up a perceptrons as the neural network

In this part we are going to set up a network and look in detail at its loss function, its activation function and how to control its topology.

> At the moment TATi can only setup multi-layer perceptrons. We may come to this in the very end.

Let's start by importing TATis's `simulation` module.

In [ ]:
import TATi.simulation as tati

The `simulation` module is an especially *light-weight*, yet powerful interface to TATi.

> Although the class name is `Simulation` (inside a module `simulation`), we will refer to it as `tati` here.

Let us take a look at its *docstring*.

In [ ]:
help(tati)

It relies on a `dict` of **options**. These options control every aspect of TATi: the network, the dataset, how and what files are written, ...

Moreover, it offers a set of functions that perform specific tasks like fitting, sampling, ...
Finally, there are a data descriptors that grant access to network internals.

At the moment, we concentrate on the options.

A specific command `tati.help()` lists all available options.

In [ ]:
tati.help()

We will not go through all of them, but let's at least take a closer look at one of them: **hidden_dimension**.

In [ ]:
tati.help("hidden_dimension")   # mind that the option name needs to be a string

There we have the option's name, a brief description, its type and the default value. Here, **hidden_dimension** has an empty list.

### Single-layer perceptron

Perceptrons have the following properties:

- input layer dimension
- output layer dimension
- number of hidden layers and their dimension
- additional drop-out layers
- activation function per node
- loss function

Each of these properties can be tuned with one of the options above.

The whole network is set up by instantiating `tati` with a given set of options. We can simply pass the options whose default value we want to change, by giving them as *keyword arguments (kwargs)* to the constructor of the class.

In [ ]:
nn = tati(input_dimension=2,
          output_dimension=1,
          hidden_dimension=0,
          hidden_activation="linear", output_activation="relu",
          loss="mean_squared")

Oops, we made a mistake! ... rats, what was again the type of **hidden_dimension**?

Of course, we knew already that it needs to be a list of ints. Then, let's fix the above instantiation.

In [ ]:
nn = tati(input_dimension=2,
          output_dimension=1,
          hidden_dimension=[0],
          hidden_activation="linear", output_activation="relu",
          loss="mean_squared")

This single-layer perceptron should have three degrees of freedom, let's check using `num_parameters()`.

In [ ]:
print(nn.num_parameters())

The *dataset* is an essential part of the network. Its dimensions define type and number of input and output nodes. Therefore, the network is internally *constructed first when a dataset is provided*.

Let us provide a dummy dataset through the option `dataset` and check again for the number of degrees of freedom.

In [ ]:
import numpy as np
# mind that features, labels need to be lists of lists
nn.dataset = [np.asarray([[0,0]], dtype=np.float32), np.asarray([[1]], dtype=np.int32)]  
print(nn.num_parameters())

That's the correct results, two weights and a single bias.

#### Inspecting the the options

In case you are curious about the options inside `tati`, use `get_options()`.

In [ ]:
print(nn.get_options())

Internally, options are stored in private variable `_options`.

> *WARNING:* Do not use `nn._options["input_dimension"]=3` directly, rather use `tati.set_options()`.

This is because some options severely affect the network topology to the effect that the network is reinstantiated. `setup_options()` takes this into account ...

In [ ]:
nn.set_options(input_dimension=3)

... and properly warns you in case the change is too severe.

#### Fixing degrees of freedom

Consider the case where we only want a network with two weights and no bias. The bias is removed if we set it to zero. How can we fix the single bias to this value?

> Fixing the bias is essentially changing the network, hence we need to add this parameter at the start. Let's reinstantiate `tati`.

When reinstantiating `tati`, the internal graph of tensorflow is reset.

In [ ]:
nn = tati(input_dimension=2,
          output_dimension=1,
          fix_parameters="output/biases/Variable:0=0.",
          hidden_dimension=[0],
          hidden_activation="linear", output_activation="relu",
          loss="mean_squared")

# mind that features, labels need to be lists of lists
nn.dataset = [np.asarray([[0,0]], dtype=np.float32), np.asarray([[1]], dtype=np.int32)]  
print(nn.num_parameters())

This is the critical change! The bias degree of freedom has been effectively removed.

The above string `output/biases/Variable:0=0.` needs some explanation. The string addresses a particular variable inside tensorflow, namely `Variable:0` in the name scopes `biases` and `output`. Moreover, we assign ("=") this variable the fixed value of *0.*. In case you want to fix a weight, replace `biases` by `weights`. In case it is the first hidden layer, user `layer1` in place of `output`. If the name cannot be found, you'll get a helpful error message. 

> There need to be as many values as the variables has components (comma-separated list). Moreover, it is not possible to fix single components. At the moment only all weights of a layer or all biases of a layer can be fixed.

### Multi-layer perceptron

Let us return to `set_options()` and changing *hidden_dimension*.

What do you do in case you really want a different network? You need to reinstantiate `tati` with the different set of options. This will automatically reset tensorflow's internal computational graph.

> Therefore, you cannot have two instances of `tati` at the same time.

Let's add two hidden layers, each with 8 nodes. Moreover, we want to use the *sigmoid* function for activation. Finally, we need to use the cross entropy function with softmax as loss.

In [ ]:
nn = tati(input_dimension=2,
          output_dimension=1,
          hidden_dimension=[8, 8],
          hidden_activation="sigmoid", output_activation="relu",
          loss="softmax_cross_entropy")

Let us see the number of degrees of freedom - note this is less tedious once you see how to pass a dataset easily.

In [ ]:
# mind that features, labels need to be lists of lists
nn.dataset = [np.asarray([[0,0]], dtype=np.float32), np.asarray([[1]], dtype=np.int32)]  
print(nn.num_parameters())

Let us briefly check whether this is true: 

In [ ]:
print(2*8+8*8+8*1+8+8+1)

Seems correct.

This is all about the setting up of the network. Next we will be looking at specifying the dataset.

### Summary

- `Simulation` module's principal design: **options**, a set of *functions*, a few data *descriptors*.
- how to get help on `Simulaton` and its set of *options*.
- a dataset is stricly necessary to actually use a neural network (lazy construction).
- how to set up a single-layer perceptron and a multi-layer perceptron.